In [1]:
import pandas as pd
from collections import Counter
df0 = pd.read_csv('imagenet-1k-256x256_gemma-2b_layer0.csv')
df6 = pd.read_csv('imagenet-1k-256x256_gemma-2b_layer6.csv')
df10 = pd.read_csv('imagenet-1k-256x256_gemma-2b_layer10.csv')
df12 = pd.read_csv('imagenet-1k-256x256_gemma-2b_layer12.csv')
# df12it = pd.read_csv('cifar100-enriched_gemma-2b-it_layer12.csv')

In [2]:
# load image_net_labels.txt as a dictionary
with open('image_net_labels.txt', 'r') as f:
    image_net_labels = eval(f.read())
image_net_labels

{0: 'tench, Tinca tinca',
 1: 'goldfish, Carassius auratus',
 2: 'great white shark, white shark, man-eater, man-eating shark, Carcharodon carcharias',
 3: 'tiger shark, Galeocerdo cuvieri',
 4: 'hammerhead, hammerhead shark',
 5: 'electric ray, crampfish, numbfish, torpedo',
 6: 'stingray',
 7: 'cock',
 8: 'hen',
 9: 'ostrich, Struthio camelus',
 10: 'brambling, Fringilla montifringilla',
 11: 'goldfinch, Carduelis carduelis',
 12: 'house finch, linnet, Carpodacus mexicanus',
 13: 'junco, snowbird',
 14: 'indigo bunting, indigo finch, indigo bird, Passerina cyanea',
 15: 'robin, American robin, Turdus migratorius',
 16: 'bulbul',
 17: 'jay',
 18: 'magpie',
 19: 'chickadee',
 20: 'water ouzel, dipper',
 21: 'kite',
 22: 'bald eagle, American eagle, Haliaeetus leucocephalus',
 23: 'vulture',
 24: 'great grey owl, great gray owl, Strix nebulosa',
 25: 'European fire salamander, Salamandra salamandra',
 26: 'common newt, Triturus vulgaris',
 27: 'eft',
 28: 'spotted salamander, Ambystoma 

In [4]:
def map_label(df):
    label = [image_net_labels[i] for i in df['fine_label']]
    df['label'] = label
    return df

df0 = map_label(df0)
df6 = map_label(df6)
df10 = map_label(df10)
df12 = map_label(df12)
# df12it = map_label(df12it)

In [5]:
def get_feature_counts(df):
    
    all_keys = []
    for features in df['retrieved_features']:
        feature_dict = eval(features)  # Convert string to dictionary
        all_keys.extend(feature_dict.keys())
        
    x = Counter(all_keys).most_common()
    print('before:',len(x))
    # now i want to remove key value > 1000 from feature_dict
    x = dict(x)
    for key in list(x.keys()):
        if x[key] > 1000:
            del x[key]
    print('after:',len(x))
    print('=====')
    return x

# 265
# 1186
# 2213
# 1823
# 2016

feature_counts0 = get_feature_counts(df0)
feature_counts6 = get_feature_counts(df6)
feature_counts10 = get_feature_counts(df10)
feature_counts12 = get_feature_counts(df12)
# feature_counts12it = get_feature_counts(df12it)

def clean_features(df, feature_counts):
    new_features = []
    for features in df['retrieved_features']:
        features = eval(features)
        features = {k:v for k,v in features.items() if k in feature_counts}
        new_features.append(features)
    df['cleaned_features'] = new_features
    return df

df0 = clean_features(df0, feature_counts0)
df6 = clean_features(df6, feature_counts6)
df10 = clean_features(df10, feature_counts10)
df12 = clean_features(df12, feature_counts12)
# df12it = clean_features(df12it, feature_counts12it)

before: 641
after: 584
=====
before: 2465
after: 2304
=====
before: 4960
after: 4651
=====
before: 4541
after: 4251
=====


In [7]:
df12

,image_id,fine_label,retrieved_features,label,cleaned_features
0,0,65,"{""1435"": ""details about the condition and feat...",sea snake,"{'4400': 'references to the word ""this.""', '13..."
1,1,970,"{""496"": ""elements related to warmth and comfor...",alp,{'1449': 'references to forests and logging-re...
2,2,230,"{""1134"": "" links or calls to action for furthe...","Shetland sheepdog, Shetland sheep dog, Shetland",{'5670': 'elements related to outdoor dining e...
3,3,809,"{""174"": ""the symbol '-' indicating lists or ne...",soup bowl,{'2865': ' references to different types of fo...
4,4,516,"{""58"": "" phrases and words related to comforta...",cradle,"{'13133': ' instances of the word ""a"" along wi..."
...,...,...,...,...,...
49995,49995,283,"{""21"": "" mentions of cats and related terms"", ...",Persian cat,"{'21': ' mentions of cats and related terms', ..."
49996,49996,26,"{""88"": ""references to insect-related issues an...","common newt, Triturus vulgaris","{'840': ' the phrase ""plenty of"" in various co..."
49997,49997,232,"{""58"": "" phrases and words related to comforta...",Border collie,{'1782': 'references to magazines and their as...
49998,49998,982,"{""1134"": "" links or calls to action for furthe...","groom, bridegroom","{'2074': 'instances of the word ""happy"" and va..."


In [8]:
# print out all above numbers out of 10000
print('layer0:',sum([i!={} for i in df0['cleaned_features']])/len(df0))
print('layer6:',sum([i!={} for i in df6['cleaned_features']])/len(df6))
print('layer10:',sum([i!={} for i in df10['cleaned_features']])/len(df10))
print('layer12:',sum([i!={} for i in df12['cleaned_features']])/len(df12))
# print('layer12it:',sum([i!={} for i in df12it['cleaned_features']])/10000)

layer0: 0.3485
layer6: 0.82518
layer10: 0.96936
layer12: 0.9688


In [9]:
df12

,image_id,fine_label,retrieved_features,label,cleaned_features
0,0,65,"{""1435"": ""details about the condition and feat...",sea snake,"{'4400': 'references to the word ""this.""', '13..."
1,1,970,"{""496"": ""elements related to warmth and comfor...",alp,{'1449': 'references to forests and logging-re...
2,2,230,"{""1134"": "" links or calls to action for furthe...","Shetland sheepdog, Shetland sheep dog, Shetland",{'5670': 'elements related to outdoor dining e...
3,3,809,"{""174"": ""the symbol '-' indicating lists or ne...",soup bowl,{'2865': ' references to different types of fo...
4,4,516,"{""58"": "" phrases and words related to comforta...",cradle,"{'13133': ' instances of the word ""a"" along wi..."
...,...,...,...,...,...
49995,49995,283,"{""21"": "" mentions of cats and related terms"", ...",Persian cat,"{'21': ' mentions of cats and related terms', ..."
49996,49996,26,"{""88"": ""references to insect-related issues an...","common newt, Triturus vulgaris","{'840': ' the phrase ""plenty of"" in various co..."
49997,49997,232,"{""58"": "" phrases and words related to comforta...",Border collie,{'1782': 'references to magazines and their as...
49998,49998,982,"{""1134"": "" links or calls to action for furthe...","groom, bridegroom","{'2074': 'instances of the word ""happy"" and va..."


In [15]:
# do a function where coarse_label is the same, check which key in cleaned_features appear the most top 5

def get_top5_features(df, coi='label'):
    top5_features = []
    labels = set(df[coi])
    for i in labels:
        df_coarse = df[df[coi] == i]
        all_keys = []
        for features in df_coarse['cleaned_features']:
            all_keys.extend(features.keys())
        # iterate all and combine into one dict
        all_features = {}
        for features in df['cleaned_features']:
            all_features.update(features)
        x = Counter(all_keys).most_common()
        top5_features.append({i:x[:5]})
        print(i, x[:5], 'This is out of:', len(df[coi])/len(labels))
        print([all_features[i[0]] for i in x[:5]])
    return top5_features

top5_features0 = get_top5_features(df0)

Bedlington terrier [('2652', 2), ('13491', 1), ('11007', 1), ('11081', 1), ('6463', 1)] This is out of: 50.050050050050054
['mentions of locations or establishments, mostly related to restaurants or entertainment venues', 'words related to requests or asking for something, specifically for support or engagement in a cause or activity', 'positive adjectives', 'mentions of different aspects of life and personal experiences', 'instances where the word "should" appears in the text']
Irish water spaniel [('4864', 2), ('8197', 1), ('15166', 1), ('2652', 1), ('2494', 1)] This is out of: 50.050050050050054
['information related to personal experiences and stories, especially involving relationships and activities', "mentions of a particular person's ranking or placement in a competition based on a specific time or score, often followed by the person who follows them in ranking or placement", 'first-person singular pronouns used in sentences', 'mentions of locations or establishments, mostly re

In [16]:
top5_features12 = get_top5_features(df12)

Bedlington terrier [('7113', 14), ('2344', 7), ('5004', 6), ('6858', 5), ('12143', 4)] This is out of: 50.050050050050054
['references to "fur" or related terms', ' terms related to knitting and knitting techniques', ' references to problems and their resolutions', ' variations of the word "shape."', 'the word "but."']
Irish water spaniel [('3674', 6), ('15610', 6), ('7236', 3), ('12414', 3), ('145', 3)] This is out of: 50.050050050050054
[' expressions indicating excitement and joy in a scientific context', 'sentences describing pet ownership and recommendations for families', 'terms related to flooding and emergency response situations', 'indications of time or duration', 'instances of expressions related to fear and anxiety in a narrative context']
reel [('6999', 35), ('9850', 19), ('6149', 16), ('12965', 15), ('15250', 8)] This is out of: 50.050050050050054
['information about fishing activity and success rates', ' mentions of fishing', ' technical issues related to engine performa

In [7]:
top5_features12it = get_top5_features(df12it)

oak_tree [('8783', 46), ('3618', 25), ('3231', 20), ('8940', 20), ('13383', 19)] This is out of: 100.0
['references to the sun', 'keywords related to agriculture and farming practices', 'information related to electronic communication, especially email addresses', 'descriptions of real estate properties', 'mentions of locations and activities outdoors']
plate [('1715', 69), ('11971', 66), ('12721', 39), ('2201', 36), ('3134', 30)] This is out of: 100.0
['references to or descriptions of circles', 'phrases related to political movements and slogans', 'details about the arrangement of molecules in ice crystals', 'information related to food and cooking recipes', 'references to specific meals at various locations']
cloud [('11234', 80), ('8783', 47), ('10242', 45), ('14644', 40), ('12721', 31)] This is out of: 100.0
['the word "cloud" in various contexts', 'references to the sun', 'references to the sky or celestial objects', 'descriptions related to celestial events and astronomical phen

In [8]:
top5_features12 = get_top5_features(df12, coi='coarse_label')

household_furniture [('5821', 306), ('6551', 262), ('58', 226), ('6859', 138), ('6338', 131)] This is out of: 500.0
[' descriptors related to furniture specifications and attributes', ' references to wood or wooden materials', ' phrases and words related to comfortable pet products', ' references to sleep and related experiences', ' references to home organization and storage solutions']
household_electrical_devices [('15389', 166), ('7574', 147), ('1146', 141), ('4382', 135), ('4870', 125)] This is out of: 500.0
['technical specifications and features of led rope lights', 'technical specifications and features related to laptop and keyboard designs', ' terms related to televisions and display technology specifications', 'terms related to household appliances and laundry equipment', ' references to time zones and clocks']
large_carnivores [('9557', 124), ('16131', 110), ('3053', 99), ('21', 89), ('4964', 87)] This is out of: 500.0
['references to dogs and related activities', 'descript

In [9]:
top5_features12it = get_top5_features(df12it, coi='coarse_label')

household_furniture [('2599', 321), ('15691', 312), ('4788', 263), ('6992', 166), ('15022', 128)] This is out of: 500.0
['mentions of timber and wood and discussions around their properties, use in construction, and comparison to other materials', 'references to technology and smart home devices', 'references to specific locations and establishments', 'references to books, book clubs, and literary activities', 'phrases related to storytelling or plot development']
household_electrical_devices [('5419', 229), ('7170', 181), ('4897', 162), ('4904', 161), ('10384', 147)] This is out of: 500.0
['technical specifications and features of electronic devices', 'technical descriptions related to products and appliances', 'descriptions related to user experience and interaction design in the context of playing games', 'references to electronic devices like cellphones', 'technical information about mechanical devices, specifically related to typewriters']
large_carnivores [('15611', 182), ('10198

trees [('8783', 153), ('3231', 92), ('8940', 86), ('8564', 77), ('3618', 72)] This is out of: 500.0
['references to the sun', 'information related to electronic communication, especially email addresses', 'descriptions of real estate properties', 'mentions of specific tv shows and locations', 'keywords related to agriculture and farming practices']
non-insect_invertebrates [('1951', 127), ('12721', 105), ('3674', 98), ('2296', 82), ('11971', 79)] This is out of: 500.0
['mentions of various types of snakes', 'details about the arrangement of molecules in ice crystals', 'descriptions of weapons and tools, particularly historical ones', 'mentions of the word "hat"', 'phrases related to political movements and slogans']
large_natural_outdoor_scenes [('8783', 217), ('752', 180), ('341', 150), ('9486', 141), ('14644', 139)] This is out of: 500.0
['references to the sun', 'information related to mountains and geographical features like ranges', 'information related to geological formations an

In [10]:
# TODO
# get these vectors, get their pre vit embedding, post vit embedding, and the difference, and top five activation patterns
# then plot them
